In [1]:
import os
import json
import praw
import requests

In [2]:
import urllib3
urllib3.disable_warnings()

In [3]:
input_file_forum_listing="forums.txt"
output_file_raw_reddit_content="reddit_content_spirituality.json"
post_limit_per_forum=1000

In [4]:
with open(input_file_forum_listing, "r") as forum_list_f:
    forum_list_content = forum_list_f.read() 
    forum_list = forum_list_content.split("\n") 
    print(forum_list) 

['spirituality', 'SpiritualAwakening', 'spiritual', 'Spiritualchills']


In [5]:
session = requests.Session()
session.verify = False

CLIENT_ID=os.getenv('REDDIT_CLIENT_ID')
CLIENT_SECRET=os.environ.get('REDDIT_CLIENT_SECRET')

USER_AGENT='your_user_agent'

reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=CLIENT_SECRET,
                     user_agent=USER_AGENT,
                     requestor_kwargs={'session': session})

In [ ]:
scrape_count = 0
questions = []
for forum in forum_list:
    forum_h = reddit.subreddit(forum)
    reddit.config.store_json_result = True
    forum_questions = []

    # Retrieve posts from the subreddit
    posts = forum_h.new(limit=post_limit_per_forum)  # Use the "new" method to retrieve new posts

    # Iterate over the posts and extract relevant information
    for post in posts:
        scrape_count = scrape_count + 1
        if scrape_count % 100 == 0:
            print(f"Processing item {scrape_count} in forum {forum}")
        entry={}
        try:
            #print(post)
            #print(dir(post))
            if post.id:
                entry["id"]=post.id
            if post.subreddit and post.subreddit.display_name:
                entry["forum"]=post.subreddit.display_name
            if post.title:
                entry["title"]=post.title
            if post.score:
                entry["score"]=post.score
            if post.author and post.author.name: 
                entry["author"]=post.author.name
            if post.selftext: 
                entry["content"]=post.selftext
            entry["created_utc"]=post.created_utc
            entry["created"]=post.created
            entry["url"]=post.url
            entry["upvote_ratio"]=post.upvote_ratio
            entry["ups"]=post.ups
            entry["downs"]=post.downs
            entry["shortlink"]=post.shortlink
            entry["permalink"]=post.permalink
            entry["view_count"]=post.view_count
            entry["comments"]=len(post.comments)
            questions.append(entry)
            forum_questions.append(entry)
        except:
            print("Encountered error")
            
    with open("out__" + forum + "___" + output_file_raw_reddit_content, "w") as outfile:
        json.dump(forum_questions, outfile, indent=5)

Processing item 100 in forum spirituality
Processing item 200 in forum spirituality
Processing item 300 in forum spirituality
Processing item 400 in forum spirituality
Processing item 500 in forum spirituality
Encountered error
Processing item 600 in forum spirituality


In [ ]:
with open(output_file_raw_reddit_content, "w") as outfile:
    json.dump(questions, outfile, indent=5)